In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
# Tool - 1 , Here out 1st Tool is Wikipedia 
api = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api)


In [3]:
pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=c725ac20d46b8070700027dbf98061a182d22be1b6e0fa5cc7d332213a5081a7
  Stored in directory: c:\users\nalla\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [7]:
wiki_tool.name

'wikipedia'

In [16]:
## Libraries needed to fetch the data from URL
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
loader_AA_TC = WebBaseLoader("https://www.aa.com/i18n/customer-service/support/legal-information.jsp")
AA_TC_data = loader_AA_TC.load()
AA_TC_fecthed_data = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(AA_TC_data)
vectordb_AA_TC = FAISS.from_documents(AA_TC_fecthed_data[:20],OllamaEmbeddings())
AA_TC_retriver = vectordb_AA_TC.as_retriever()
AA_TC_retriver

C:\Users\nalla\AppData\Local\Temp\ipykernel_10736\2380442911.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb_AA_TC = FAISS.from_documents(AA_TC_fecthed_data[:20],OllamaEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001767D20FC70>, search_kwargs={})

In [18]:
loader_AA_CC = WebBaseLoader("https://www.aa.com/i18n/customer-service/support/conditions-of-carriage.jsp")
AA_CC_data = loader_AA_CC.load()
AA_CC_fetched_data = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(AA_CC_data)
vectordb_AA_CC = FAISS.from_documents(AA_CC_fetched_data[:20],OllamaEmbeddings())
AA_CC_retriver = vectordb_AA_CC.as_retriever()
AA_CC_retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001767D22D930>, search_kwargs={})

In [20]:
tools = [wiki_tool,AA_TC_retriver,AA_CC_retriver]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\nalla\\anaconda3\\envs\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001767D20FC70>, search_kwargs={}),
 VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001767D22D930>, search_kwargs={})]

In [22]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama2")
llm

OllamaLLM(model='llama2')

In [26]:
## This is an optional code to show how to import predefined froms that arae already in a structured and are present in lanchain hub
from langchain import hub

prompt_demo = hub.pull("hwchase17/openai-functions-agent")
prompt_demo.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Act as a refund agent who always helps its customer to get the maximum refund from the context.
Many customers have no idea, so read the context (Terms and Conditions of American Airlines)
and help the customer on what to talk to customer care to get the maximum refund.
You will get 10 percent of every successful refund.

<context>
{context}
</context>

{agent_scratchpad}

Question: {input}
""")


In [33]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

ValueError: Unsupported function

tags=['FAISS', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001767D20FC70> search_kwargs={}

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.